In [1]:
import ee
import os
import re
import csv
import glob
import geemap
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import statsmodels.api as sm
from scipy.stats import norm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Section

In [6]:
output_dir = 'G:/001Project/Output/Sheets/cleaned_Results/irrigation_wUse/RQ_2/Section/'

# Read the CSV file
df = pd.read_csv('G:/001Project/Output/Sheets/cleaned_Results/combined/Combined_PRISM_2021.csv')

# Convert 'Date' column to datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Filter the data for May to September
df_may_to_sep = df[(df['Date'].dt.month >= 5) & (df['Date'].dt.month <= 9)]

# Group the data by 'LN_ID' and calculate the sum of 'DAYMET' and 'PRISM' for each group
grouped_section = df_may_to_sep.groupby('LN_ID').agg(
    COUNTY=('COUNTY', 'first'),
    STATE=('STATE', 'first'),
    CROP_DIV=('CROP_DIV', 'first'),
    DOM_CROP=('DOM_CROP', 'first'),
    CLIMATE=('CLIMATE', 'first'),
    **{f'{model}_DAYMET': (model, lambda x: x.sum()) for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']},
    PR_DAYMET=('DAYMET', lambda x: x.sum()),
    **{f'{model}_PRISM': (model, lambda x: x.sum()) for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']},
    PR_PRISM=('PRISM', lambda x: x.sum())
)

# Calculate the irrigation water use (IWU) for each ET model by subtracting 'PR_DAYMET' and 'PR_PRISM' from the corresponding ET model column
for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']:
    iwu_column_daymet = f'{model}_DAYMET'
    iwu_column_prism = f'{model}_PRISM'
    grouped_section[iwu_column_daymet] -= grouped_section['PR_DAYMET']
    grouped_section[iwu_column_prism] -= grouped_section['PR_PRISM']

# Rearrange the columns to put "PR_DAYMET" and "PR_PRISM" at the end
columns_order = [col for col in grouped_section.columns if col not in ['PR_DAYMET', 'PR_PRISM']] + ['PR_DAYMET', 'PR_PRISM']
grouped_section = grouped_section[columns_order]
    
# Drop the 'PR_DAYMET' and 'PR_PRISM' columns if necessary
#grouped_section = grouped_section.drop(['PR_DAYMET', 'PR_PRISM'], axis=1)

# Save the updated dataframe back to the CSV file
grouped_section.to_csv(output_dir + 'Section_IWU_2021.csv')

## Calculate IWU for one year

In [14]:
%%time

output_dir = 'G:/001Project/Output/Sheets/cleaned_Results/irrigation_wUse/RQ_2/Section/'

# Read the CSV file
df = pd.read_csv('G:/001Project/Output/Sheets/cleaned_Results/combined/Combined_PRISM_2021.csv')

# Convert 'Date' column to datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Filter the data for May to September
df_may_to_sep = df[(df['Date'].dt.month >= 5) & (df['Date'].dt.month <= 9)]

# Group the data by 'LN_ID' and calculate the sum of precipitation products for each group
grouped_section = df_may_to_sep.groupby('LN_ID').agg(
    LN_ID=('LN_ID', 'first'),
    COUNTY=('COUNTY', 'first'),
    STATE=('STATE', 'first'),
    CROP_DIV=('CROP_DIV', 'first'),
    DOM_CROP=('DOM_CROP', 'first'),
    CLIMATE=('CLIMATE', 'first'),
    **{f'{model}_DAYMET': (model, lambda x: x.sum()) for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']},
    PR_DAYMET=('DAYMET', lambda x: x.sum()),
    **{f'{model}_gridMET': (model, lambda x: x.sum()) for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']},
    PR_gridMET=('gridMET', lambda x: x.sum()),
    **{f'{model}_IMERG': (model, lambda x: x.sum()) for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']},
    PR_IMERG=('IMERG', lambda x: x.sum()),
    **{f'{model}_NLDAS': (model, lambda x: x.sum()) for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']},
    PR_NLDAS=('NLDAS', lambda x: x.sum()),
    **{f'{model}_PRISM': (model, lambda x: x.sum()) for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']},
    PR_PRISM=('PRISM', lambda x: x.sum())
)

# Calculate the irrigation water use (IWU) for each ET model by subtracting precipiration from the corresponding ET model column
for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']:
    iwu_column_daymet = f'{model}_DAYMET'
    iwu_column_gridmet = f'{model}_gridMET'
    iwu_column_imerg = f'{model}_IMERG'
    iwu_column_nldas = f'{model}_NLDAS'
    iwu_column_prism = f'{model}_PRISM'
    grouped_section[iwu_column_daymet] -= grouped_section['PR_DAYMET']
    grouped_section[iwu_column_gridmet] -= grouped_section['PR_gridMET']
    grouped_section[iwu_column_imerg] -= grouped_section['PR_IMERG']
    grouped_section[iwu_column_nldas] -= grouped_section['PR_NLDAS']
    grouped_section[iwu_column_prism] -= grouped_section['PR_PRISM']

# Rearrange the columns in alphabetical order, including 'LN_ID'
columns_order = ['LN_ID'] + [col for col in grouped_section.columns
                            if col != 'LN_ID' and col not in ['PR_DAYMET', 'PR_gridMET', 'PR_IMERG', 'PR_NLDAS', 'PR_PRISM']] + \
                            ['PR_DAYMET', 'PR_gridMET', 'PR_IMERG', 'PR_NLDAS', 'PR_PRISM']
grouped_section = grouped_section[columns_order]
    
# Drop the 'PR_DAYMET' and 'PR_PRISM' columns if necessary
#grouped_section = grouped_section.drop(['PR_DAYMET', 'PR_PRISM'], axis=1)

# Save the updated dataframe back to the CSV file
output_file = os.path.join(output_dir + 'Section_IWU_2021.csv')

# Save the updated dataframe to the output file
grouped_section.to_csv(output_file, index=False)

print("IWU computed and saved to '{}'.".format(output_file))

IWU computed and saved to 'G:/001Project/Output/Sheets/cleaned_Results/irrigation_wUse/RQ_2/Section/Section_IWU_2021.csv'.
CPU times: total: 3min 31s
Wall time: 4min 7s


## Loop through all the years and compute IWU

In [18]:
%%time

# List of years
years_to_process = [2016, 2017, 2018, 2019, 2020, 2021]

# Output directory
output_dir = 'G:/001Project/Output/Sheets/cleaned_Results/irrigation_wUse/RQ_2/Section/'

for year in years_to_process:
    # Read the CSV file for the current year
    file_path = f'G:/001Project/Output/Sheets/cleaned_Results/combined/Combined_PRISM_{year}.csv'
    df = pd.read_csv(file_path, parse_dates=['Date'])

    # Convert 'Date' column to datetime type
    df['Date'] = pd.to_datetime(df['Date'])

    # Filter the data for May to September (modify for crop calendar)
    df_may_to_sep = df[(df['Date'].dt.month >= 5) & (df['Date'].dt.month <= 9)]

    # Group the data by 'LN_ID' and calculate the sum of precipitation products for each group
    grouped_section = df_may_to_sep.groupby('LN_ID').agg(
        LN_ID=('LN_ID', 'first'),
        COUNTY=('COUNTY', 'first'),
        STATE=('STATE', 'first'),
        CROP_DIV=('CROP_DIV', 'first'),
        DOM_CROP=('DOM_CROP', 'first'),
        CLIMATE=('CLIMATE', 'first'),
        **{f'{model}_DAYMET': (model, 'sum') for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']},
        PR_DAYMET=('DAYMET', 'sum'),
        **{f'{model}_gridMET': (model, 'sum') for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']},
        PR_gridMET=('gridMET', 'sum'),
        **{f'{model}_IMERG': (model, 'sum') for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']},
        PR_IMERG=('IMERG', 'sum'),
        **{f'{model}_NLDAS': (model, 'sum') for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']},
        PR_NLDAS=('NLDAS', 'sum'),
        **{f'{model}_PRISM': (model, 'sum') for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']},
        PR_PRISM=('PRISM', 'sum')
    )

    # Calculate the irrigation water use (IWU) for each ET model
    for model in ['DISALEXI', 'EEMETRIC', 'GEESEBAL', 'PTJPL', 'SIMS', 'SSEBOP']:
        for product in ['DAYMET', 'gridMET', 'IMERG', 'NLDAS', 'PRISM']:
            iwu_column = f'{model}_{product}'
            grouped_section[iwu_column] -= grouped_section[f'PR_{product}']

    # Rearrange the columns in alphabetical order, including 'LN_ID' and keep the precipitation products at the end.
    columns_order = ['LN_ID'] + [col for col in grouped_section.columns
                                if col != 'LN_ID' and col not in ['PR_DAYMET', 'PR_gridMET', 'PR_IMERG', 'PR_NLDAS', 'PR_PRISM']] + \
                                ['PR_DAYMET', 'PR_gridMET', 'PR_IMERG', 'PR_NLDAS', 'PR_PRISM']
    grouped_section = grouped_section[columns_order]

    # Save the updated dataframe to a CSV file for the current year
    output_file = os.path.join(output_dir, f'Section_IWU_{year}.csv')
    grouped_section.to_csv(output_file, index=False)

    print(f"IWU computed and saved to '{output_file}' for {year}.")

print("Done!")

IWU computed and saved to 'G:/001Project/Output/Sheets/cleaned_Results/irrigation_wUse/RQ_2/Section/Section_IWU_2016.csv' for 2016.
IWU computed and saved to 'G:/001Project/Output/Sheets/cleaned_Results/irrigation_wUse/RQ_2/Section/Section_IWU_2017.csv' for 2017.
IWU computed and saved to 'G:/001Project/Output/Sheets/cleaned_Results/irrigation_wUse/RQ_2/Section/Section_IWU_2018.csv' for 2018.
IWU computed and saved to 'G:/001Project/Output/Sheets/cleaned_Results/irrigation_wUse/RQ_2/Section/Section_IWU_2019.csv' for 2019.
IWU computed and saved to 'G:/001Project/Output/Sheets/cleaned_Results/irrigation_wUse/RQ_2/Section/Section_IWU_2020.csv' for 2020.
IWU computed and saved to 'G:/001Project/Output/Sheets/cleaned_Results/irrigation_wUse/RQ_2/Section/Section_IWU_2021.csv' for 2021.
Done!
CPU times: total: 37.7 s
Wall time: 54.8 s


### Aggregate by section

In [9]:
output_dir = 'G:/001Project/Output/Sheets/cleaned_Results/irrigation_wUse/RQ_2/Section/'

# Read the CSV file
df = pd.read_csv('G:/001Project/Output/Sheets/cleaned_Results/combined/Combined_data_2021.csv')

# Convert 'Date' column to datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Filter the data for May to September
df_may_to_sep = df[(df['Date'].dt.month >= 5) & (df['Date'].dt.month <= 9)]


# Group the data by 'LN_ID'
grouped_section = df_may_to_sep.groupby('LN_ID').agg(
    COUNTY=('COUNTY', 'first'),
    STATE=('STATE', 'first'),
    CROP_DIV=('CROP_DIV', 'first'),
    DOM_CROP=('DOM_CROP', 'first'),
    CLIMATE=('CLIMATE', 'first'),
    **{f'IWU_{model}_PRISM': (model, lambda x: x.sum()) for model in ['DISALEXI_ET', 'EEMETRIC_ET', 'GEESEBAL_ET', 'PTJPL_ET', 'SIMS_ET', 'SSEBOP_ET']},
    PR_PRISM=('PRISM', lambda x: x.sum())
)

# Calculate the irrigation water use by subtracting PRISM from each ET model
for model in ['DISALEXI_ET', 'EEMETRIC_ET', 'GEESEBAL_ET', 'PTJPL_ET', 'SIMS_ET', 'SSEBOP_ET']:
    iwu_column = f'IWU_{model}_PRISM'
    grouped_section[iwu_column] -= grouped_section['PR_PRISM']

#grouped_section = grouped_section.drop(['IWU_PRISM'], axis=1)

# Save the updated dataframe back to the CSV file
#grouped_section.to_csv(output_dir + 'Section_IWU_2021.csv')

In [52]:
grouped_section

COUNTY     STATE  CROP_DIV                 DOM_CROP CLIMATE  \
LN_ID                                                                    
2401      Lincoln  Colorado         2  Dbl Crop WinWht/Sorghum     BSk   
2413      Prowers  Colorado        11                  Alfalfa     BSk   
2416         Weld  Colorado        14                     Corn     BSk   
2422     Sedgwick  Colorado         4                     Corn     BSk   
2424       Morgan  Colorado        14                  Alfalfa     BSk   
...           ...       ...       ...                      ...     ...   
2701140    Yoakum     Texas         6                   Cotton     BSk   
2701142    Yoakum     Texas         2                   Cotton     BSk   
2701143    Yoakum     Texas         2                   Cotton     BSk   
2701144    Yoakum     Texas         2                   Cotton     BSk   
2701145    Yoakum     Texas         2                   Cotton     BSk   

         IWU_DISALEXI_ET_PRISM  IWU_EEMETRIC_ET_PRISM  IWU_GEESEBAL_ET_PRISM  \
LN_ID                                                                          
2401                131.784499             126.049692              70.639346   
2413                141.410742             260.781522             155.735531   
2416                179.630874             170.473123             211.423795   
2422                325.314169             316.031727             223.228996   
2424                311.633481             373.696241             313.111411   
...                        ...                    ...                    ...   
2701140              57.637242             192.540542              62.312045   
2701142             -33.629480              89.308994             -28.760176   
2701143             -84.050049              23.365132             -88.311861   
2701144             142.039172             326.848090             191.581633   
2701145             -22.435323              52.040923             -37.306937   

         IWU_PTJPL_ET_PRISM  IWU_SIMS_ET_PRISM  IWU_SSEBOP_ET_PRISM  \
LN_ID                                                                 
2401             124.561495         286.505242           117.142108   
2413             249.586778         411.124947           219.208875   
2416             215.288593         239.739960           154.687155   
2422             238.765405         398.953532           314.454148   
2424             329.805761         470.900119           392.343638   
...                     ...                ...                  ...   
2701140          192.956430         291.778958            39.879910   
2701142          100.168257         172.585298           -91.240991   
2701143           50.277710         144.668458          -134.570002   
2701144          185.967797         286.355244           174.339765   
2701145          109.220940         180.804723           -69.316796   

           PR_PRISM  
LN_ID                
2401     317.700685  
2413     317.700685  
2416     317.700685  
2422     317.700685  
2424     317.700685  
...             ...  
2701140  372.305889  
2701142  372.305889  
2701143  372.305889  
2701144  372.305889  
2701145  372.305889  

[114872 rows x 12 columns]

### Agrregate by County

In [ ]:
output_dir = 'G:/001Project/Output/Sheets/cleaned_Results/irrigation_wUse/RQ_2/County/'

# Read the CSV file
df = pd.read_csv('G:/001Project/Output/Sheets/cleaned_Results/combined/Combined_data_2016.csv')

# Convert 'Date' column to datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Filter the data for May to September
df_may_to_sep = df[(df['Date'].dt.month >= 5) & (df['Date'].dt.month <= 9)]


# Group the data by 'LN_ID'
grouped_county = df_may_to_sep.groupby('COUNTY').agg(
    STATE=('STATE', 'first'),
    CROP_DIV=('CROP_DIV', 'first'),
    DOM_CROP=('DOM_CROP', 'first'),
    CLIMATE=('CLIMATE', 'first'),
    **{f'IWU_{model}_PRISM': (model, lambda x: x.sum()) for model in ['DISALEXI_ET', 'EEMETRIC_ET', 'GEESEBAL_ET', 'PTJPL_ET', 'SIMS_ET', 'SSEBOP_ET']},
    IWU_PRISM=('PRISM', lambda x: x.sum())
)

# Calculate the irrigation water use by subtracting PRISM from each ET model
for model in ['DISALEXI_ET', 'EEMETRIC_ET', 'GEESEBAL_ET', 'PTJPL_ET', 'SIMS_ET', 'SSEBOP_ET']:
    iwu_column = f'IWU_{model}_PRISM'
    grouped_county[iwu_column] -= grouped_county['IWU_PRISM']

#grouped_county = grouped_county.drop(['IWU_PRISM'], axis=1)

model_columns = ['IWU_DISALEXI_ET_PRISM', 'IWU_EEMETRIC_ET_PRISM', 'IWU_GEESEBAL_ET_PRISM',
                 'IWU_PTJPL_ET_PRISM', 'IWU_SIMS_ET_PRISM', 'IWU_SSEBOP_ET_PRISM']

# Clip off values where ET is less than 0 and assign the values to zero
grouped_county[model_columns] = grouped_county[model_columns].clip(lower=0)

# Compute the coefficient of variation (CV) for each LN_ID
grouped_county['CV'] = grouped_county[model_columns].std(axis=1) / grouped_county[model_columns].mean(axis=1)

# Save the updated dataframe back to the CSV file
#grouped_county.to_csv(output_dir + 'County_IWU_2016.csv')

In [ ]:
grouped_county

### Aggregate by State

In [ ]:
output_dir = 'G:/001Project/Output/Sheets/cleaned_Results/irrigation_wUse/RQ_2/State/'

# Read the CSV file
df = pd.read_csv('G:/001Project/Output/Sheets/cleaned_Results/combined/Combined_data_2016.csv')

# Convert 'Date' column to datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Filter the data for May to September
df_may_to_sep = df[(df['Date'].dt.month >= 5) & (df['Date'].dt.month <= 9)]


# Group the data by 'LN_ID'
grouped_state = df_may_to_sep.groupby('STATE').agg(
    CROP_DIV=('CROP_DIV', 'first'),
    DOM_CROP=('DOM_CROP', 'first'),
    CLIMATE=('CLIMATE', 'first'),
    **{f'IWU_{model}_PRISM': (model, lambda x: x.sum()) for model in ['DISALEXI_ET', 'EEMETRIC_ET', 'GEESEBAL_ET', 'PTJPL_ET', 'SIMS_ET', 'SSEBOP_ET']},
    IWU_PRISM=('PRISM', lambda x: x.sum())
)

# Calculate the irrigation water use by subtracting PRISM from each ET model
for model in ['DISALEXI_ET', 'EEMETRIC_ET', 'GEESEBAL_ET', 'PTJPL_ET', 'SIMS_ET', 'SSEBOP_ET']:
    iwu_column = f'IWU_{model}_PRISM'
    grouped_state[iwu_column] -= grouped_state['IWU_PRISM']

#grouped_state = grouped_state.drop(['IWU_PRISM'], axis=1)

model_columns = ['IWU_DISALEXI_ET_PRISM', 'IWU_EEMETRIC_ET_PRISM', 'IWU_GEESEBAL_ET_PRISM',
                 'IWU_PTJPL_ET_PRISM', 'IWU_SIMS_ET_PRISM', 'IWU_SSEBOP_ET_PRISM']

# Clip off values where ET is less than 0 and assign the values to zero
grouped_state[model_columns] = grouped_state[model_columns].clip(lower=0)

# Compute the coefficient of variation (CV) for each LN_ID
grouped_section['CV'] = grouped_state[model_columns].std(axis=1) / grouped_state[model_columns].mean(axis=1)

# Save the updated dataframe back to the CSV file
#grouped_state.to_csv(output_dir + 'State_IWU_2016.csv')

In [ ]:
grouped_state